# Reinforcement Learning Final Project #

### Caleb Stillman ###

GitHub: https://github.com/cstillma/RLFinalProject

## Project Overview ## 

In this final project for the CSPB3202 Intro to AI course, I'll be building a Deep Q-Network agent to test in the Box2D Car Racing environment available through OpenAI Gymnasium. This particular reinforcement learning algorithm has been covered in lectures but was not one that was used in any previous homeworks, so this will serve as a hands-on test of my understanding of this algorithm. I will also be including a simple random agent so I have a performance baseline to compare my DQN model to in the Car Racing environment. Time permitting, I will test the DQN agent on other OpenAI Gymnasium environments like the Lunar Landing environment to see the algorithm's performance in different settings. 

## Approach ##

### Setting up the environment ###

In [1]:
import gymnasium as gym
import random_agent


### Setting a Baseline ###

I'll be using a simple random agent to set a baseline for the performance of the DQN agent.

In [2]:
# Call the main function for the random agent
random_agent.RA_main()

Episode 1 finished with a score of reward: -582.9149253731705
Episode 2 finished with a score of reward: -1051.8576642337243
Episode 3 finished with a score of reward: -1150.3259259260644
Episode 4 finished with a score of reward: -1090.3798165139176
Episode 5 finished with a score of reward: -1134.9443037976214


The testing the random agent for 5 episodes shows that it performs very poorly at this game. Not a hard benchmark to beat at least. 

## Result ##

## Video Demo ##

## Conclusion ##